In [ ]:
import torch.nn.functional as F
from torch.nn import init
from math import radians, cos, sin, asin, sqrt
import numpy as np
import torch.optim as optim
from torch import nn
import torch
import random
#import matplotlib.pyplot as plt
import math
from math import sqrt

import pandas as pd
#网络结构
class TGCN(nn.Module):
    def __init__(self):
        super(TGCN, self).__init__()
        self.hidden_size = 64
        self.reset_parameters()
        self.stations = 35
        self.features = 29
        self.forecast = 6
        self.historical = 12
        self.tem = 3
        self.wf = nn.Parameter(torch.FloatTensor(self.hidden_size, hidden_size))
        self.bf = nn.Parameter(torch.FloatTensor(self.hidden_size))
        self.a = nn.Parameter(torch.FloatTensor(1))
        self.encoder = nn.Parameter(torch.FloatTensor(self.features, self.hidden_size))
        self.bencoder = nn.Parameter(torch.FloatTensor(self.hidden_size))
        self.decoder = nn.Parameter(torch.FloatTensor(self.hidden_size, self.hidden_size))
        self.bdecoder = nn.Parameter(torch.FloatTensor(self.hidden_size))
        self.Z1 = nn.Parameter(torch.FloatTensor(self.hidden_size*2, self.hidden_size))
        self.Ws = nn.Parameter(torch.FloatTensor(self.hidden_size, self.hidden_size))
        self.Wp = nn.Parameter(torch.FloatTensor(1, self.hidden_size))
        self.Wgcn = nn.Parameter(torch.FloatTensor(self.hidden_size, self.hidden_size))
        self.Wgcn1 = nn.Parameter(torch.FloatTensor(self.hidden_size, self.hidden_size))
        self.Wgcn2 = nn.Parameter(torch.FloatTensor(self.hidden_size, self.hidden_size))
        self.Wgcn3 = nn.Parameter(torch.FloatTensor(self.hidden_size, self.hidden_size))
        self.Wgcn4 = nn.Parameter(torch.FloatTensor(self.hidden_size, self.hidden_size))
        self.con1 = nn.Conv2d(self.hidden_size, self.hidden_size, (self.tem, 1), padding=(1, 0))
        self.con2 = nn.Conv2d(self.hidden_size, self.hidden_size, (self.historical, 1))
        self.averagecon = nn.Conv2d(self.hidden_size, self.hidden_size, (self.historical, 1))
        self.wshort = nn.Parameter(torch.FloatTensor(self.stations, self.stations))
        self.wc = nn.Parameter(torch.FloatTensor(self.hidden_size*2, self.hidden_size))
        self.dcon1 = nn.ConvTranspose2d(self.hidden_size, self.hidden_size, (1, self.stations))
        self.wfusion = nn.Parameter(torch.FloatTensor(self.hidden_size*2, self.hidden_size))
        self.wout = nn.Parameter(torch.FloatTensor(self.hidden_size, 1))
        self.avgwout = nn.Parameter(torch.FloatTensor(self.hidden_size, 1))
        self.reset_parameters()

    def reset_parameters(self):
        for weight in self.parameters():
            init.uniform_(weight, -0.1, 0.1)

    #输入数据格式转化
    def input_transform(self, x):
        local_inputs, labels = x
        local_inputs = local_inputs.permute(1, 0, 2, 3) #(12h, batch, features, stations)
        labels = labels.permute(1, 0, 2, 3) #(6h, batch, 1features, stations)
        n_input_encoder = local_inputs.data.size(2) #28features
        batch_size = local_inputs.data.size(1)
        _local_inputs = local_inputs.contiguous().view(-1, n_input_encoder, self.stations) #(12*batch,features,stations)
        _local_inputs = torch.split(_local_inputs, batch_size, 0) #12个数组 每个数组（batch，stations，features）
        encoder_inputs = _local_inputs
        _labels = labels.contiguous().view(-1, self.stations, 1)
        _labels = torch.split(_labels, batch_size, 0) #6个数组 （batch, 1, stations）
        _lastinp = local_inputs[11:12, :, 0:1, :]  # 1, 256, 1, 35
        _lastinp = _lastinp.contiguous().view(-1, 1, self.stations)
        _lastinp = torch.split(_lastinp, batch_size, 0)
        decoder_inputs = list(_lastinp) + list(_labels[:-1])  # 6*(256, 1, 35)
        return encoder_inputs, _labels, decoder_inputs

    #第一层lstm
    def SpatialGCN(self, encoder_inputs, decoder_inputs, Ws, teather_ratio):
        Inputs = encoder_inputs #12 (batch, stations, m)
        batch_size = Inputs[0].data.size(0)
        stations = Inputs[0].data.size(2)
        temporalM = torch.FloatTensor(batch_size, self.historical, stations, self.hidden_size)
        memory_cell = torch.rand(stations, self.hidden_size).to(device)
        index = 0
        for flinput in Inputs: #n个时间步
            dec = torch.FloatTensor(batch_size, stations, self.hidden_size).to(device)
            for s in range(flinput.data.size(2)):
                en_out = torch.matmul(flinput[:, :, s].float(),self.encoder) + self.bencoder
                de_out = torch.matmul(en_out, self.decoder) + self.bdecoder
                dec[:, s:s+1, :] = de_out.reshape(batch_size, 1, self.hidden_size)
                #dec[:, s:s+1, :] =  flinput[:, :, s:s+1].float().permute(0, 2, 1)
            input_ = torch.cat([dec.to(device), memory_cell.reshape(1, stations, self.hidden_size).repeat(batch_size,1,1)], 2)  
            memory_cell = self.a * memory_cell +(1-self.a) * torch.matmul(torch.mean(dec,0), self.wf) + self.bf
            input_ = torch.matmul(input_, self.Z1)
            #input_ = dec.to(device)
            top = torch.exp(torch.sigmoid(torch.matmul(torch.matmul(input_, Ws), input_.permute(0, 2, 1))))
            bottom = torch.sum(top, dim=1)  # (m,1)
            S = torch.div(top, bottom.reshape(batch_size, stations,1))
            I = torch.eye(self.stations, self.stations).to(device)
            A = S + I
            sumA = torch.sum(A, dim=1)
            A_ = torch.div(A, sumA.reshape(batch_size, stations,1))
            Gcn = torch.matmul(input_, self.Wgcn)
            Gcn = torch.matmul(A_, Gcn) #batch,n,hidden_size
            Gcn1 = torch.matmul(input_, self.Wgcn1)
            Gcn1 = torch.matmul(A_ **2, Gcn1) #batch,n,hidden_size
            temporalM[:, index, :, :] = torch.matmul(torch.cat([Gcn, Gcn1], dim=2), self.wfusion)         
            index = index+1
        predictlist=[]
        averagelist = []
        for dint in decoder_inputs:          
            averageT = torch.mean(temporalM, dim=2, keepdim=True)
            averagecon = self.averagecon(averageT.permute(0, 3, 1, 2).to(device)) #batch,hiddensize,1,1
            predict_avg = torch.matmul(averagecon.reshape(batch_size, self.hidden_size), self.avgwout)
            averagelist.append(predict_avg)
            con1 = self.con1(temporalM.permute(0, 3, 1, 2).to(device)) #batch,hiddensize,12,35
            con2 = self.con2(con1.to(device)).permute(0, 2, 3, 1) #batch,hiddensize,1,1
            short_input = temporalM[:, -1, :, :].permute(0, 2, 1)
            cshort = torch.matmul(short_input.to(device), self.wshort).permute(0, 2, 1).reshape(batch_size, 1, stations, self.hidden_size) #batch, stations, hidden
            con = torch.matmul(torch.cat([con2, cshort], dim=3), self.wc)
            #con = cshort
            is_teather = random.random() < teather_ratio
            w = self.wout.permute(1, 0)
            tout = torch.matmul(dint.reshape(batch_size, stations, 1).float(), w).reshape(batch_size, 1, stations, self.hidden_size)
            out = (tout if is_teather else con).to(device)
            temporalM = torch.cat([temporalM[:, 1:, :, :].to(device), out], dim=1)
            output = con.reshape(batch_size, hidden_size, stations).permute(0, 2, 1)
            predict = torch.matmul(output, self.wout)
            predictlist.append(predict)
        return predictlist,averagelist

    def forward(self, x, teather_ratio):
        encoder_inputs, labels, decoder_inputs = self.input_transform(x)
        predicts, avgs = self.SpatialGCN(encoder_inputs, decoder_inputs, self.Ws, teather_ratio)
        # predicts = self.Decoder(As, Af, At, encoder_inputs, h1list, h2list, h3list, self.dcell1, self.dcell2, self.dcell3)
        return predicts, avgs, labels


In [ ]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""

    def __init__(self, patience=7, verbose=False, delta=0):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement.
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score <= self.best_score + self.delta + 0.000001:
            self.counter += 1
            print('EarlyStopping counter:', self.counter)
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print('Validation loss min:', self.val_loss_min)
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss

def get_batch_feed_dict(k, batch_size, training_data, train_labels_data):
    batch_train_inp = training_data[k:k + batch_size]
    batch_label_inp = train_labels_data[k:k + batch_size]
    feed_dict = (batch_train_inp, batch_label_inp)
    return feed_dict

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
np.random.seed(2021)
random.seed(2021)
torch.manual_seed(2021)
hidden_size = 64
feature_num = 29
model = TGCN()
model.to(device)
import datetime


print('开始时间-------', datetime.datetime.now())
total_epoch = 500
batch_size = 256
lr = 0.0001

traindata = np.load("train.npy")
testdata = np.load("test.npy")
pm25_train = np.load("pm25_train.npy")
pm25_test = np.load("pm25_test.npy")

max_value = 560
min_value = 2

optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08)
index = [i for i in range(len(testdata))]
random.shuffle(index)
testdata = testdata[index]
pm25_test = pm25_test[index]

def criterion(preds, avgs, labels, para):
    loss_fn = nn.MSELoss()
    loss = 0.0
    for ps, avg, ls in zip(preds, avgs, labels):
        ls_avg = torch.mean(ls, dim=1, keepdim=False)
        loss = loss + loss_fn(ps.cpu().float(), ls.cpu().float())+ para*loss_fn(avg.cpu().float(), ls_avg.cpu().float())
    return loss

num_train = 7777
num_valid = 1193
num_test = 1193

train_data = torch.from_numpy(traindata[:num_train, :, :, :]).to(device)  # 6990, 12, 34, 26
train_labels_data = torch.from_numpy(pm25_train[:num_train, :, :, :]).to(device)  # 6990, 6, 1, 26

valid_data = torch.from_numpy(traindata[num_train:, :, :, :]).to(device)
valid_labels_data = torch.from_numpy(pm25_train[num_train:, :, :]).to(device)

test_data = torch.from_numpy(testdata[:, :, :, :]).to(device)
test_labels_data = torch.from_numpy(pm25_test[:, :, :]).to(device)

# 64,12,34,26    64,6,1,26

train_losses = []
valid_losses = []
avg_train_losses = []
avg_valid_losses = []

early_stopping = EarlyStopping(patience=15, verbose=True)

teather_ratio=0.5
te = 0.0
k=0.5
para = 0.3
for i in range(total_epoch):
    print('----------epoch {}-----------{}'.format(i, datetime.datetime.now()))
    i += 1
    teather_ratio = teather_ratio * (0.9 ** (i // 30))
    for j in range(0, num_train, batch_size):
        x = get_batch_feed_dict(j, batch_size, train_data, train_labels_data)
        preds,avgs, labels = model(x,teather_ratio)
        optimizer.zero_grad()
        loss = criterion(preds, avgs, labels, para)
        loss.backward(retain_graph=True)
        optimizer.step()
        train_losses.append(loss.item())

    for j in range(0, num_valid, batch_size):
        x = get_batch_feed_dict(j, batch_size, valid_data, valid_labels_data)
        preds,avgs, labels = model(x,te)
        loss = criterion(preds,avgs, labels, para)
        valid_losses.append(loss.item())
    train_loss = np.average(train_losses)
    valid_loss = np.average(valid_losses)
    avg_train_losses.append(train_loss)
    avg_valid_losses.append(valid_loss)

    early_stopping(valid_loss, model)
    print_msg = (' train_loss:' + str(train_loss) + ' valid_loss:' + str(valid_loss))
    print(print_msg)
    if early_stopping.early_stop:
        print("Early stopping")
        break


# test
def MAEloss(preds, labels):
    loss_fn = nn.L1Loss()
    loss = 0.0
    for ps, ls in zip(preds, labels):
        loss = loss + loss_fn(ps.cpu().float() * (max_value - min_value) + min_value,
                                  ls.cpu().float() * (max_value - min_value) + min_value)
    return loss


def RMSEloss(preds, labels):
    loss_fn = nn.MSELoss()
    loss = 0.0
    for ps, ls in zip(preds, labels):
        loss = loss + loss_fn(ps.cpu().float() * (max_value - min_value) + min_value,
                                  ls.cpu().float() * (max_value - min_value) + min_value)
    return loss


rmses = []
maes = []
test_loss = []
test_loss1 = []
for k in range(0, num_test, batch_size):
    x = get_batch_feed_dict(k, batch_size, test_data, test_labels_data)
    preds,avgs, labels= model(x,te)
    loss = RMSEloss(preds, labels) / 6
    loss1 = MAEloss(preds, labels) / 6
    test_loss.append(loss.item())
    test_loss1.append(loss1.item())

print('===============METRIC===============')
MAE = np.average(test_loss1)
mseLoss = math.sqrt(np.average(test_loss))
print('MAE = {:.6f}'.format(MAE))
print('RMSE = {:.6f}'.format(mseLoss))